In [1]:
import psycopg2
import numpy as np
import pandas as pd

In [2]:
import plotly.express as px
import plotly.graph_objs as go

In [3]:
from datetime import datetime, time

In [4]:
import copy

In [5]:
host="192.168.2.23"
port=5432
user="amt"
dbname="qdap_test"

In [2]:
import sys
sys.stdout

In [6]:
def make_connection_to_db(host, port, user, dbname):
    conn = psycopg2.connect(host= host, port=port, user=user, dbname=dbname)
    cursor = conn.cursor()
    return cursor, conn

In [7]:
def fetch_options_data_timeframe(cursor, symbol, expiry_type, start_date, end_date):
    cursor.execute(
        f'''
            SELECT *
            FROM ohlcv_options_per_minute oopm
            WHERE symbol = '{symbol}' 
            AND oopm.expiry_type = '{expiry_type}'
            AND oopm.date_timestamp >= '{start_date}'
            AND oopm.date_timestamp <= '{end_date}'
            ORDER BY date_timestamp ASC;
        '''
    )
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df

In [8]:
def fetch_futures_data_timeframe(cursor, symbol, expiry_type, start_date, end_date):
    query = f'''
        SELECT *
        FROM ohlcv_future_per_minute ofpm
        WHERE ofpm.symbol = '{symbol}'
        AND ofpm.expiry_type = '{expiry_type}'
        AND date_timestamp >= '{start_date}'
        AND date_timestamp <= '{end_date}'
        ORDER BY date_timestamp ASC;
    '''
    cursor.execute(query)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df

In [9]:
def fetch(host, port, user, dbname, symbols, expiry_type_futures, expiry_type_options, start_date, end_date):
    cursor, conn = make_connection_to_db(host, port, user, dbname)
    dictionary_futures = {}
    dictionary_options = {}
    for symbol in symbols:
        df_futures = fetch_futures_data_timeframe(cursor, symbol, expiry_type_futures, start_date, end_date)
        df_futures['date_timestamp'] = pd.to_datetime(df_futures['date_timestamp'])
        df_futures['expiry'] = pd.to_datetime(df_futures['expiry'])
        
        df_options = fetch_options_data_timeframe(cursor, symbol, expiry_type_options, start_date, end_date)
        df_options['date_timestamp'] = pd.to_datetime(df_options['date_timestamp'])
        df_options['expiry'] = pd.to_datetime(df_options['expiry'])
        
        df_options = df_options[df_options['date_timestamp'].dt.date == df_options['expiry'].dt.date]
        expiries = pd.to_datetime(df_options['expiry']).dt.date
        df_futures = df_futures[df_futures['date_timestamp'].dt.date.isin(expiries)]
        
        dictionary_futures[symbol] = df_futures
        dictionary_options[symbol] = df_options
    cursor.close()
    conn.close()
    return dictionary_futures, dictionary_options

In [10]:
symbols = ["BANKNIFTY", "NIFTY", "FINNIFTY", "MIDCPNIFTY"]
start_date = '2023-09-06'
end_date = '2024-05-01'
expiry_type_options = 'IW1'
expiry_type_futures = 'I'
moneyness_strike = 0
fund_locked = 1000 # inr
fund_locked *= 100
transaction_cost = 11.5
slippage = 10

In [11]:
DICT_FUTURES, DICT_OPTIONS = fetch(host, port, user, dbname, symbols, expiry_type_futures, expiry_type_options, start_date, end_date)

In [12]:
dict_futures = DICT_FUTURES.copy()
dict_options = DICT_OPTIONS.copy()

In [13]:
def fill_df_and_get_continuous_excluding_market_holidays(df, market_holidays_csv_path = r'C:\Users\user4\Desktop\exchange_holidays.csv'):
    # reading the market holidays from the list provided to us
    df = df.copy()
    market_holidays_df = pd.read_csv(market_holidays_csv_path, parse_dates=['holiday_date'])
    market_holidays = market_holidays_df['holiday_date'].dt.date.tolist()
    # generating a range of all the dates that exists from the first date to the last date
    start_date = df['date_timestamp'].dt.date.iloc[0]
    end_date = df['date_timestamp'].dt.date.iloc[-1]
    all_days = pd.date_range(start=start_date, end=end_date, freq='B')
    # mask for the invalid days 
    trading_holidays = all_days.to_series().apply(lambda x: x.date() in market_holidays)
    trading_days = all_days[~trading_holidays]
    # Generate a complete range of the 375 trading minutes for each trading day
    trading_minutes = pd.date_range(start='09:15:00', end='15:29:00', freq='min').time
    # Create a complete index of trading timestamps
    complete_index = pd.DatetimeIndex([pd.Timestamp.combine(day, time) for day in trading_days for time in trading_minutes])
    df = df.set_index('date_timestamp')
    try:
        df = df.reindex(complete_index).ffill()
    except:
        pass
    return df, complete_index

In [14]:
def process_parse_futures(df_futures):
    df_futures = df_futures.copy()
    # dropping duplicate entries
    df_futures = df_futures.drop_duplicates(subset='date_timestamp', keep='first')
    # required information
    info_needed = ["high", "low", "open", "close", "date_timestamp"]
    df_futures = df_futures[info_needed]
    # made continuous data if there were some discontinuity in the available data
    _, complete_index = fill_df_and_get_continuous_excluding_market_holidays(df_futures)
    df_futures = df_futures.set_index('date_timestamp')
    df_futures = df_futures.reindex(complete_index).ffill()
    return df_futures

In [15]:
def convert(df, complete_index, ohlc):
    df = df.copy()
    df = df.pivot(columns='strike', values=ohlc)
    df = df.reindex(complete_index).ffill()
    return df

In [16]:
def process_parse_options(df_options):
    df_options = df_options.copy()
    info_needed = ['low', 'high', 'open', 'close']
    # dropping duplicate entries
    df_options = df_options.drop_duplicates(subset=['date_timestamp', 'strike', 'opt_type'], keep='first')
    # processing calls
    df_calls = df_options[(df_options['opt_type'] == 'CE')]
    _, complete_index = fill_df_and_get_continuous_excluding_market_holidays(df_calls)
    df_calls = df_calls.set_index('date_timestamp')
    df_calls = [convert(df_calls, complete_index, info) for info in info_needed]
    # ease of access of a calls open close as a function of timestamp and strike
    # tracking all the existing strikes that were available for the calls
    call_strikes = np.array(df_calls[0].columns, dtype=int)
    # processing puts
    df_puts  = df_options[(df_options['opt_type'] == 'PE')]
    _, complete_index = fill_df_and_get_continuous_excluding_market_holidays(df_puts)
    df_puts = df_puts.set_index('date_timestamp')
    # ease of access of a puts open close as a function of timestamp and strike
    df_puts = [convert(df_puts, complete_index, info) for info in info_needed]
    # tracking all the existing strikes that were available for the puts
    put_strikes = np.array(df_puts[0].columns, dtype=int)
    return df_puts, df_calls, [put_strikes, call_strikes]

In [17]:
# def filter_futures_data_for_options(symbols, dict_futures, dict_options):
#     master_df = {}
#     for symbol in symbols:
#         df_futures = dict_futures[symbol]
#         df_options = dict_options[symbol]
#         df_futures = process_parse_futures(df_futures)
#         df_puts, df_calls, [strikes_put, strikes_call] = process_parse_options(df_options)
#         # df_symbol_callsclose_futures = df_calls_close.join(df_futures, how='left')
#         # df_symbol_putsclose_futures = df_puts_close.join(df_futures, how='left')
#         # df_symbol = [df_symbol_putsclose_futures, df_symbol_callsclose_futures]
#         df_symbol = [df_puts, df_calls, df_futures]
#         master_df[symbol] = df_symbol
#     return master_df

In [18]:
def get_common_strikes(symbols, dict_options):
    dict_options = copy.deepcopy(dict_options)
    strikes = {}
    for symbol in symbols:
        df_options = dict_options[symbol]
        _, _, [put_strikes, call_strikes] = process_parse_options(df_options)
        common_strikes = set(put_strikes).intersection(set(call_strikes))
        common_strikes = sorted(list(common_strikes))
        common_strikes = pd.Series(common_strikes)
        strikes[symbol] = common_strikes
    return strikes

In [19]:
def get_synthetic_futures(symbols, dict_options, dict_futures):
    dict_options = copy.deepcopy(dict_options)
    dict_futures = copy.deepcopy(dict_futures)
    common_strikes_symbols = get_common_strikes(symbols, dict_options)
    ohlc_list = ['open', 'high', 'low', 'close']
    for symbol in symbols:
        df_futures = dict_futures[symbol]
        df_options = dict_options[symbol]
        # Process and parse futures and options data
        df_futures = process_parse_futures(df_futures)
        df_calls, df_puts, all_strikes = process_parse_options(df_options)
        common_strikes = np.array(common_strikes_symbols[symbol])  # Ensure common_strikes is a NumPy array
        # Initialize synthetic columns with 0
        df_futures[['synthetic_' + ohlc for ohlc in ohlc_list]] = 0
        for time_index in range(df_puts[0].shape[0]):
            for ohlc_i, ohlc in enumerate(ohlc_list):
                future_price = df_futures[ohlc].iloc[time_index]
                # Find the closest strike index to the futures price
                ix = np.argmin(np.abs(common_strikes - future_price))
                c_minus_p = np.inf
                synthetic_future = None
                # Search within a range of moneyness
                for moneyness in range(max(ix - 10, 0), min(ix + 10, len(common_strikes))):
                    strike = common_strikes[moneyness]
                    diff = df_calls[ohlc_i].iloc[time_index][strike] - df_puts[ohlc_i].iloc[time_index][strike]
                    if diff < c_minus_p:
                        c_minus_p = diff
                        synthetic_future = strike + c_minus_p
                # Assign the synthetic future value
                df_futures.at[df_futures.index[time_index], 'synthetic_' + ohlc] = synthetic_future
        dict_futures[symbol] = df_futures
    return dict_futures

In [20]:
dict_futures_with_synthetic = get_synthetic_futures(symbols, dict_options, dict_futures)

IndexError: single positional indexer is out-of-bounds

In [ ]:
def ATR(df_futures, period=14):
    df_futures = df_futures.copy()
    df_futures['previous_close'] = df_futures['close'].shift(1)
    df_futures['tr1'] = df_futures['high'] - df_futures['low']
    df_futures['tr2'] = (df_futures['high'] - df_futures['previous_close']).abs()
    df_futures['tr3'] = (df_futures['low'] - df_futures['previous_close']).abs()
    df_futures['true_range'] = df_futures[['tr1', 'tr2', 'tr3']].max(axis=1)
    # Calculate the ATR (using the Exponential Moving Average)
    df_futures['atr'] = df_futures['true_range'].ewm(span=period, adjust=False).mean()
    # Drop the intermediate columns used for calculation
    df_futures.drop(columns=['previous_close', 'tr1', 'tr2', 'tr3', 'true_range'], inplace=True)
    return df_futures

In [ ]:
df_futures = dict_futures['BANKNIFTY']
df_options = dict_options['BANKNIFTY']
# Process and parse futures and options data
df_futures = process_parse_futures(df_futures)
df_calls, df_puts, all_strikes = process_parse_options(df_options)